In [ ]:
import pandas as pd
import requests
from string import Template
import time
import hmac
import hashlib

In [ ]:
VERSION = '1.1'
BASE_REQUEST = Template('https://bittrex.com/api/v$version/$scope/')

PUBLIC_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='public')
MARKET_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='market')
ACCOUNT_BASE_URL = BASE_REQUEST.substitute(version=VERSION, scope='account')

# Read public and secret keys from keys.txt
API_KEY = open('keys.txt', 'r').read().split('\n')[0].split(':')[1].strip()
SECRET_KEY = open('keys.txt', 'r').read().split('\n')[1].split(':')[1].strip()

PUBLIC_METHODS = ['getmarkets', 'getcurrencies', 'getticker', 'getmarketsummaries', 'getmarketsummary', 'getorderbook',
                  'getmarkethistory']

MARKET_METHODS = ['buylimit', 'selllimit', 'cancel', 'getopenorders']

ACCOUNT_METHODS = ['getbalances', 'getbalance', 'getdepositaddress', 'withdraw', 'getorder', 'getorderhistory', 
                   'getwithdrawalhistory', 'getdeposithistory']

In [118]:
# Method for unpacking arguments and forming them into a formatted string that can be used in a Bittrex API request 
def unpack_args(args):
    formatted_args = ''
    
    if args != None:
        for k in args.keys():
            formatted_args += (k + '=' + args[k]+ '&')
            
    return formatted_args


# Wrapper method for accessing the Bittrex API
def bittrex_request(scope, method, args=None):

    if scope == PUBLIC_BASE_URL:
        request_url_template = Template('$scope$method?$args')
        
        if args is not None:
            request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args))
        else:
            request_url = request_url_template.substitute(scope=scope, method=method, args='')

        return pd.DataFrame(requests.get(request_url).json()['result'])

    elif scope == MARKET_BASE_URL:
        request_url_template = Template('$scope$method?$args$auth')
        
        nonce = time.time()
        auth = {'apikey': str(API_KEY), 'nonce': str(nonce)}
        
        if args is not None:
            request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args), auth=unpack_args(auth))
        else:
            request_url = request_url_template.substitute(scope=scope, method=method, args='', auth=unpack_args(auth))
        
        # Sign the request
        sign = hmac.new(SECRET_KEY, request_url, hashlib.sha512).hexdigest()

        # Add the sign to the request header
        request_headers = {'apisign': sign}
        
        # Make the request
        return pd.DataFrame(requests.get(request_url, headers = request_headers).json()['result'])
    
    elif scope == ACCOUNT_BASE_URL:
        request_url_template = Template('$scope$method?$args$auth')
        
        nonce = time.time()
        auth = {'apikey': str(API_KEY), 'nonce': str(nonce)}
        
        if args is not None:
            request_url = request_url_template.substitute(scope=scope, method=method, args=unpack_args(args), auth=unpack_args(auth))
        else:
            request_url = request_url_template.substitute(scope=scope, method=method, args='', auth=unpack_args(auth))
        
        # Sign the request
        sign = hmac.new(SECRET_KEY, request_url, hashlib.sha512).hexdigest()

        # Add the sign to the request header
        request_headers = {'apisign': sign}

        # Make the request
        return pd.DataFrame(requests.get(request_url, headers = request_headers).json()['result'])

## Public API request example

args = {'type': 'sell', 'market': 'BTC-XRP'}
bittrex_request(PUBLIC_BASE_URL, 'getorderbook', args)

## Account API request example

bittrex_request(ACCOUNT_BASE_URL, 'getorderhistory')

## Market API request example

bittrex_request(MARKET_BASE_URL, 'getopenorders')